## Removing M-Relief part from our modified overlapping multisurf

In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from scipy.spatial.distance import pdist, squareform
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import pairwise_distances as pair_dist
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.neighbors import NearestNeighbors
import pandas as pd
import math

In [2]:
def check_numeric(X):
  newX = np.array(X).reshape(-1)
  return all(not isinstance(n, str) for n in newX)

# X=np.array([[6,7,8],[3,0,5]])
# print(check_numeric(X))
# print(X)
# print(check_numeric([1,2,3]))

In [3]:
def error_return(p):
  ranked = np.arange(p, dtype=int) 
  weight = np.empty((1,p))
  weight = np.squeeze(weight) 
  weight[:] = np.nan
  return ranked, weight


### Preprocessing data and call feature selection algorithm

In [4]:
def Modified_MultiSurf_configure(X,Y, **kwargs):
  X = np.array(X)
  Y = np.array(Y)
  prior, numUpdates, categoricalX, kernelWidth = kwargs['prior'], kwargs['numUpdates'], kwargs['categoricalX'], kwargs['kernelWidth']

  # if not check_numeric(X):
  #   print('X does not contain numeric data')
  #   p = X.shape[1] # no of attributes
  #   return error_return(p) 

  # Check if the input sizes are consistent
  if Y.shape[0] != X.shape[0]:
    print('number of instances and output labels doesnot match')
    p = X.shape[1] # no of attributes
    return error_return(p)
  
  # converting classes as 0 to #classes
  [Y, grp] = pd.factorize(Y)

  # grpToInd contains class to index mapping, grpToInd[className]= ind
  grpToInd={}
  for ind, g in enumerate(grp):
    grpToInd[g]= ind

  # # removing incomplete instances
  # df_XY = pd.DataFrame(X)
  # df_XY['Y'] = Y
  # df_XY = df_XY.dropna()
  # X, Y = np.array(df_XY.iloc[:, 0:-1]), np.array(df_XY.iloc[:, -1])
  
  Ngrp = len(grp)
  N = X.shape[0]
  C = np.zeros((N,Ngrp))
  C[np.arange(N), Y] = True
  
  # Get class probs
  if not prior or prior == 'empirical':
    classProb = C.sum(0)
  elif prior == 'uniform':
    classProb = np.ones((1, Ngrp))
  elif isinstance(prior, dict):
    classProb = np.zeros((1,Ngrp))
    # if prior is a dictionary of class and prior prob
    if not len(prior):
      print('prior dictionrary is empty')
      p = X.shape[1] # no of attributes
      return error_return(p)
    for g, prob in prior.items():
      classProb[grpToInd[g]]=prob
  elif check_numeric(prior):
    if len(prior)!= Ngrp or any(p <0 for p in prior) or not all(isinstance(p, float) for p in prior) or all(p == 0 for p in prior) :
      print('prior doesnot fulfil all conditions')
      p = X.shape[1] # no of attributes
      return error_return(p)
    classProb = prior
  else:
    print('prior is invalid')
    p = X.shape[1] # no of attributes
    return error_return(p)
  
  #  Normalize class probs
  classProb = classProb/classProb.sum()
  classProb = np.squeeze(classProb) 
  #  If there are classes with zero probs, remove them
  zeroProb = classProb != 0
  
  t = [ not classProb[y]==0 for y in Y] # contains array of size [Ngrp,1] with values [True, True, False ...]
  X = X[t]
  Y = Y[t]
  C = C[t]
  C = C[:,zeroProb]
  classProb = classProb[zeroProb]

  #  Do we have enough observations?
  if len(Y)<2:
    print('not enough instances')
    p = X.shape[1] # no of attributes
    return error_return(p)

  # if not check_numeric([K]) or K<0:
  #   print('invalid value of K')
  #   p = X.shape[1] # no of attributes
  #   return error_return(property)
  # K= math.ceil(K)

  # Check number of updates
  if (not numUpdates=='all') and (not check_numeric([numUpdates]) or numUpdates<=0):
    print('numUpdates is invalid')
  elif (not numUpdates) or numUpdates=='all':
    numUpdates = X.shape[0]
  else:
    numUpdates = math.ceil(numUpdates)
    
  # Check the type of categoricalX
  if not categoricalX or (categoricalX != 'on' and categoricalX != 'off'):
      print('categoricalX is invalid')
  categoricalX = (categoricalX == 'on')   

  # Find max and min for every predictor
  p = X.shape[1] # no of attributes
  Xmax = X.max(0)
  Xmin = X.min(0) 
  Xdiff = Xmax - Xmin
  Xmean = np.mean(X, axis=0) 

  # Exclude single-valued attributes
  isDiffValue = Xdiff >= 1e-9  # boolean array of size #attributes [1,0,0,0]
  if not any(isDiffValue):
    p = X.shape[1] # no of attributes
    return error_return(p)
  
  X = X[:, isDiffValue ]
  Xmax = Xmax[isDiffValue ] 
  Xmin = Xmin[isDiffValue ]  
  Xdiff = Xdiff[isDiffValue] 
  Xmean = Xmean[isDiffValue]
  rejected = [ i for i in range(len(isDiffValue)) if not isDiffValue[i]]  # indices of the deleted attributes (values range from 1 to p)
  accepted = [ i for i in range(len(isDiffValue)) if isDiffValue[i]]  # indices of remaining attributes (values range from 1 to p)

  # Scale and center the attributes
  if not categoricalX:
      X = (X - Xmin) / Xdiff 

  # The #updates cannot be more than the #observations
  numUpdates = min(numUpdates, X.shape[0])

  # Call MultiSurf. By default all weights are set to NaN.
  weight = np.empty(p) 
  weight[:] = np.nan

  weight[accepted] = Modified_MultiSurf(X, Y, C,classProb,numUpdates,categoricalX, kernelWidth) 

  # Assign ranks to attributes
  sorted = np.argsort(-weight[accepted])
  accepted = np.array(accepted)
  ranked = accepted[sorted]
  ranked = np.append(ranked, rejected)
  ranked = ranked.astype(int)

  return ranked, weight 



# Overlapping_MultiSurf algorithm function

In [5]:
def Modified_MultiSurf(scaledX, scaledY, C,classProb,numUpdates, categoricalX, kernelWidth):
  
  # MultiSurf for classification
  numInstances,numAttr = scaledX.shape 
  attrWeights = np.zeros(numAttr) 
  # C is boolean 2D matrix of size (N, Ngrp) i.e. (Number of instances vs Number of classes)
  numClasses = C.shape[1]   # Ngrp

  # Choose the random instances
  np.random.seed(123) 
  rndIdx = np.random.choice(numInstances, numUpdates, replace=False)   # random indices of m instances from total of n instances
  idxVec = np.arange(numInstances)   # [0 .. N-1]

   # Make searcher objects, one object per class. 
  instIdxPerClass = {}  
  for c in range(numClasses):
    c_C = C[:,c]
    instIdxPerClass[c] = np.array([i for i in range(len(c_C)) if c_C[i]], dtype=int)   # instances of class c
  
  # selecting distance function
  distFunc = 'manhattan'
  if categoricalX:
    distFunc = 'hamming'
  else: 
    distFunc = 'manhattan'
  
  pairwiseDistance = pair_dist(scaledX, metric=distFunc)

   # Outer loop, for updating attribute weights iteratively
  for i in range(numUpdates):
    targetIdx = rndIdx[i]
    
      # Choose the correct random observation
    targetInst = scaledX[targetIdx,:]

      # Find the class for this observation
    targetC = C[targetIdx, :]
    thisC = scaledY[targetIdx] # taking the class of target instance
    
    # find nearest hits and misses
    allInstIdx = np.arange(numInstances)
    otherIdxExceptTarget = allInstIdx[allInstIdx!=targetIdx] 
    meanDistance = np.mean(pairwiseDistance[targetIdx][otherIdxExceptTarget])
    stdDistance = np.std(pairwiseDistance[targetIdx][otherIdxExceptTarget])
    threshold = meanDistance - (stdDistance/2.0)
    # print('threshold : ' ,threshold)
    Hits = np.array([], dtype = 'int')
    allMissesInd = np.array([], dtype = 'int')
    Misses = {}
    
    for j in range(numInstances): 
      newC = scaledY[j]
      if pairwiseDistance[targetIdx][j] < threshold:
        if newC == thisC and j != targetIdx:
          Hits = np.append(Hits, j)
        elif newC != thisC:
          allMissesInd =  np.append(allMissesInd, j)
          if newC in Misses:
            Misses[newC] = np.append(Misses[newC], j)
          else:
            Misses[newC] = np.array([j], dtype= 'int')

    neighbourIndicesFromOtherClass = {}
    for c in range(numClasses):
      neighbourIndicesFromOtherClass[c] = np.append(allMissesInd[ scaledY[allMissesInd] != c ], np.append(Hits, targetIdx))

    # Misses = np.array(Misses)
    # print('targetID :',targetIdx)
    # print('HitId :', Hits)
    # print('MissId :', Misses)
    
    #***************** ATTRIBUTE UPDATE *****************************
    # Inner loop to update weights for each attribute:

    # print('targetID :',targetIdx)
    # print("target :",targetInst)
    # print('HitId :', Hits)
    # print("Hit :", X[Hits])
    # print('MissId :', Misses[:, 0:-1])
    # print("Miss :", X[Misses[:, 0:-1]])
    
    for j in range(numAttr):
      dH = diffH(j,scaledX,targetIdx,Hits, allMissesInd, categoricalX, kernelWidth)/numUpdates
      # print('dH', dH) 
      dM = diffM(j,scaledX,targetIdx, Misses, neighbourIndicesFromOtherClass, len(allMissesInd), categoricalX, classProb, kernelWidth)/numUpdates
      # print('dM', dM)
      attrWeights[j] = attrWeights[j] - dH  + dM 
      
      #****************************************************************
    
    # print('attrWeights : ', attrWeights)
  
  return attrWeights    

In [6]:
def cityblock(thisX, X):
  d = abs(X - thisX) 
  return d

In [7]:
def hamming(thisX, X):
  d = (X != thisX).astype(int) 
  return d

In [8]:
def kernel_func(d, kernelWidth):
  return np.exp(-d/kernelWidth)

In [9]:
#--------------------------------------------------------------------------
# DIFFH (for Modified_Overlapping_MultiSurfClass): Function to calculate difference measure
# for an attribute between the selected instance and its hits

def diffH(a, X, targetIdx, Hits, missesInd, categoricalX, kernelWidth):

  # print(a, targetIdx, Hits, categoricalX)

  # If no hits, return zero by default
  if not len(Hits):
      return 0

  distFunc = hamming if categoricalX else cityblock 

  if len(missesInd) <= 0:
    distMeas = np.ones(len(Hits)) * distFunc(X[targetIdx, a], X[Hits,a])
    return distMeas.sum()/ len(Hits)


  # Calculate weighted sum of distances

  distMeas = 0
  totalPNotInOverlapping = 0

  distMetric = 'manhattan'
  if categoricalX:
    distMetric = 'hamming'
  else: 
    distMetric = 'manhattan'

  disttoMissesFromHits = np.sum(kernel_func(pair_dist(np.expand_dims(X[Hits, a], axis = 1), np.expand_dims(X[missesInd,a], axis = 1), metric = distMetric), kernelWidth), axis = 1)
  
  distToOtherHits = pair_dist(np.expand_dims(X[np.append(Hits, targetIdx), a], axis = 1), metric = distMetric)
  distToOtherHits = distToOtherHits[~np.eye(distToOtherHits.shape[0], dtype = bool)].reshape(distToOtherHits.shape[0],-1)
  distToOtherHits = distToOtherHits[:-1]
  distToOtherHits = np.sum(kernel_func(distToOtherHits, kernelWidth), axis = 1)
  pNotInOverlapping = 1 -  disttoMissesFromHits/ (disttoMissesFromHits + distToOtherHits)

  distMeas = pNotInOverlapping * distFunc(X[targetIdx, a], X[Hits,a])
  distMeas = distMeas.sum()

  totalPNotInOverlapping = pNotInOverlapping.sum()

  if totalPNotInOverlapping>0:
    return distMeas/totalPNotInOverlapping
  else:
   return distMeas



In [10]:
#--------------------------------------------------------------------------
# DIFFM (for Modified_Overlapping_MultiSurfClass) : Function to calculate difference measure
# for an attribute between the selected instance and its misses
def diffM(a, X, targetIdx, Misses, neighbourIndicesFromOtherClass, totalMisses, categoricalX, classProb, kernelWidth):
  
  # print(a, targetIdx, Misses, categoricalX)

  # print('missesInd = ', missesInd)

  # If no hits, return zero by default
  if not totalMisses:
      return 0.0

  # Calculate weighted sum of distances
  distFunc = hamming if categoricalX else cityblock 

  distMeas = 0.0
  totalPNotInOverlapping = 0.0

  distMetric = 'manhattan'
  if categoricalX:
    distMetric = 'hamming'
  else: 
    distMetric = 'manhattan'

  for cls, missIdx in Misses.items():
    missIdx = missIdx.astype(int)
    
    if len(missIdx) > 0:
      pNotInOverlapping = np.ones(len(missIdx))

      if len(neighbourIndicesFromOtherClass[cls]) > 0:
        distToOtherClassIndices = np.sum(kernel_func(pair_dist(np.expand_dims(X[missIdx, a], axis = 1), np.expand_dims(X[neighbourIndicesFromOtherClass[cls],a], axis = 1), metric = distMetric), kernelWidth), axis = 1)
        
        distToOtherMissesFromSameClass = pair_dist(np.expand_dims(X[missIdx, a], axis = 1), metric = distMetric)
        distToOtherMissesFromSameClass = distToOtherMissesFromSameClass[~np.eye(distToOtherMissesFromSameClass.shape[0], dtype = bool)].reshape(distToOtherMissesFromSameClass.shape[0],-1)
        distToOtherMissesFromSameClass = np.sum(kernel_func(distToOtherMissesFromSameClass, kernelWidth), axis = 1)
        
        pNotInOverlapping = 1 - distToOtherClassIndices/(distToOtherClassIndices + distToOtherMissesFromSameClass) 
    
      distMeas += np.sum(pNotInOverlapping * distFunc(X[targetIdx, a], X[missIdx,a]) * (len(missIdx)/totalMisses) * len(Misses))
      totalPNotInOverlapping += pNotInOverlapping.sum()

  if totalPNotInOverlapping>0:
    return distMeas/totalPNotInOverlapping
  else:
   return distMeas

In [11]:
# #--------------------------------------------------------------------------
# # DIFFH (for Modified_MultiSurfClass): Function to calculate difference measure
# # for an attribute between the selected instance and its hits

# def diffH(a, X, targetIdx, Hits, Misses, categoricalX, kernelWidth):

#   # print(a, targetIdx, Hits, categoricalX)

#   missesInd = np.array([], dtype = 'int')
#   for indices in Misses.values():
#     missesInd =  np.append(missesInd, indices)

#   missesInd = missesInd.astype(int)

#   # If no hits, return zero by default
#   if not len(Hits):
#       return 0

#   # Calculate weighted sum of distances
#   distFunc = hamming if categoricalX else cityblock 

#   distMeas = 0
#   totalPNotInOverlapping = 0
  

#   for hit in Hits:
#     otherHits = np.append(Hits[Hits != hit], targetIdx)
#     totalNeighbours = np.append(otherHits, missesInd)
#     # pNotInOverlapping is the probability of a neighbour not being in an overlapping area
#     pNotInOverlapping = 1
#     if len(missesInd)>0:
#       pNotInOverlapping = 1 - np.sum(kernel_func(distFunc(X[hit, a], X[missesInd,a]), kernelWidth)) / np.sum(kernel_func(distFunc(X[hit, a], X[totalNeighbours,a]), kernelWidth))
#     distMeas += pNotInOverlapping * distFunc(X[targetIdx, a], X[hit,a])
#     totalPNotInOverlapping += pNotInOverlapping

#   # print('For attribute ',a,' Hit distmeas : ', distMeas/len(Hits))

#   if totalPNotInOverlapping>0:
#     return distMeas/totalPNotInOverlapping
#   else:
#    return distMeas


In [12]:
# #--------------------------------------------------------------------------
# # DIFFM (for Modified_MultiSurfClass) : Function to calculate difference measure
# # for an attribute between the selected instance and its misses
# def diffM(a, X, targetIdx, Hits, Misses, categoricalX, classProb, kernelWidth):
  
#   # print(a, targetIdx, Misses, categoricalX)

#   missesInd = np.array([], dtype='int')
#   for indices in Misses.values():
#     missesInd =  np.append(missesInd, indices)

#   missesInd = missesInd.astype(int)

#   # print('missesInd = ', missesInd)

#   # If no hits, return zero by default
#   if not len(missesInd):
#       return 0

#   # Calculate weighted sum of distances
#   distFunc = hamming if categoricalX else cityblock 

#   distMeas = 0
#   totalPNotInOverlapping = 0

#   totalMisses = 0.0
#   for missIdx in Misses.values():
#     totalMisses += len(missIdx)

#   Hits = np.append(Hits, targetIdx)
#   for missIdx in Misses.values():
#     missIdx = missIdx.astype(int)
#     for miss in missIdx:
#       otherMisses = missIdx[missIdx != miss]
#       totalNeighbours = np.append(missesInd[missesInd!=miss], Hits).astype(int)
#       otherClassInd = np.array([ind for ind in totalNeighbours if ind not in otherMisses], dtype= 'int')
      
#       pNotInOverlapping = 1
#       if len(otherClassInd)>0:
#         pNotInOverlapping = 1 - np.sum(kernel_func(distFunc(X[miss, a], X[otherClassInd,a]), kernelWidth)) / np.sum(kernel_func(distFunc(X[miss, a], X[totalNeighbours,a]), kernelWidth))
      

#       # Explanation: first we are calculating pNotInOverlapping * distFunc(X[targetIdx, a], X[miss,a]) for each instance in a class.
#       # So now we have the measurement in  all instance of a single class scale.
#       # Then we take the summation of  probability of each class and the sum value of pNotInOverlapping * distFunc(X[targetIdx, a], X[miss,a]) of that class,
#       #  that means pNotInOverlapping * distFunc(X[targetIdx, a], X[miss,a]) * (len(missIdx)/totalMisses) .
#       # Now we have measuremnts for a single class and all instance for it(suppose). Then we multiply it by number of Miss classes, len(Misses) and rescale the measurement 
#       # for all classes andd all miss instances.
#       # Finally, we devide the measurement by totalPNotInOverlapping, which bring it back in one instance scale.
      
#       distMeas += pNotInOverlapping * distFunc(X[targetIdx, a], X[miss,a]) * (len(missIdx)/totalMisses) * len(Misses)
#       totalPNotInOverlapping += pNotInOverlapping

#   # print('For attribute ',a,' Misses distmeas : ', distMeas/len(Misses))
#   # if totalPNotInOverlapping==0:
#   #   print('totalPNotInOverlapping = ',totalPNotInOverlapping)
#   if totalPNotInOverlapping>0:
#     return distMeas/totalPNotInOverlapping
#   else:
#    return distMeas

### Run feature selection

In [13]:
# from sklearn.datasets import load_iris, load_digits, load_wine
# X, Y = load_digits(return_X_y= True)
# print('dataset: digits')
# ranked, weight = Modified_MultiSurf_configure(X,Y,K=10,prior='uniform', numUpdates='all', categoricalX='off')
# print(ranked, weight)

In [14]:
# X = np.array([[0,0,0,0,1,0,0],
#               [0,1,0,1,0,1,0],
#               [1,0,0,0,1,1,0],
#               [0,0,0,1,0,0,0],
#               [1,1,0,0,0,0,0],
#               [0,1,0,0,1,0,0],
#               [0,0,0,0,0,1,0],
#               [1,0,0,1,0,0,0],
#               [1,0,1,0,0,0,1],
#               [1,1,0,0,1,0,1],
#               [0,0,0,0,0,0,1],
#               [1,0,1,1,1,1,1],
#               [1,0,0,0,0,0,0],
#               [1,0,0,1,0,1,1],
#               [0,1,1,0,0,0,0],
#               [0,0,1,1,0,0,1],
#               [1,1,0,1,0,0,0],
#               [1,0,1,0,1,0,1],
#               [0,1,1,1,0,0,1],
#               [1,1,1,1,1,1,1]])

# Y = np.array([0,1,1,0,0,1,0,1,1,0,0,1,1,1,1,0,0,1,1,0])

# Overlapping_MultiSurf_configure(X,Y,K=3,prior='uniform', numUpdates='all', categoricalX='on')


In [15]:
# X = np.array([[1,0,1],
#               [1,0,0],
#               [0,1,1],
#               [0,1,0],
#               [0,0,1],
#               [0,0,0],
#               [1,1,1],
#               [1,1,0]])

# Y = np.array([1,1,1,1,0,0,0,0])
# ranked, weight= Modified_MultiSurf_configure(X,Y,K=1,prior='uniform', numUpdates='all', categoricalX='on')

# print('ranked')
# print(ranked)
# print('weight')
# print(weight)


In [16]:
# def permute(x):
#   if len(x) == 7:
#     global X
#     global Y
#     Y.append((x[0] ^ x[6]))
#     X.append(x)
#     return
#   x = np.append(x,0)
#   permute(x)
#   x = x[0:-1]
#   x = np.append(x,1)
#   permute(x)

In [17]:
# X = []
# Y = []
# x = np.array([], dtype=int)
# permute(x)
# X = np.array(X)
# Y = np.array(Y)
# ranked, weight=Overlapping_MultiSurf_configure(X,Y,prior='uniform', numUpdates=len(Y)/2, categoricalX='on',kernelWidth = 3)

# print('ranked')
# print(ranked)
# print('weight')
# print(weight)


In [18]:
# !pip install skrebate
# from skrebate import ReliefF, MultiSURF, SURF, SURFstar
# import time

# # test classification dataset
# from sklearn.datasets import make_classification
# # define dataset
# X, Y = make_classification(n_samples=100, n_features=15, n_informative=10, n_redundant=0,shuffle=False, random_state=1)
# # summarize the dataset
# print(X.shape, Y.shape)
# start = time.time()
# ranked, weight= Modified_MultiSurf_configure(X,Y,prior='uniform', numUpdates='all', categoricalX='off', kernelWidth = 3)
# print("time : ", time.time()-start)

# print('ranked')
# print(ranked)
# print('weight')
# print(weight)

# # From ReBATE Framework - ReliefF
# reliefF = ReliefF(n_features_to_select=min(X.shape[1]-50, 50), n_neighbors= 10)
# reliefF.fit(X, Y)
# weight = reliefF.feature_importances_
# ranked = reliefF.top_features_

# print('ranked')
# print(ranked)
# print('weight')
# print(weight)

# # From ReBATE Framework - Modified_MultiSurf
# MultiSURF = MultiSURF(n_features_to_select=min(X.shape[1]-50, 50))
# MultiSURF.fit(X, Y)
# weight = MultiSURF.feature_importances_
# ranked = MultiSURF.top_features_


# print('ranked')
# print(ranked)
# print('weight')
# print(weight)

In [19]:
# (100, 15) (100,)
# time :  5.170177459716797
# ranked
# [ 7  6  3  0  1  2  8  5  4  9 11 13 12 14 10]
# weight
# [ 0.01882981  0.01721425  0.01122548  0.04091904  0.00299551  0.00354692
#   0.04682125  0.07100074  0.00710316 -0.00638363 -0.02656315 -0.01504503
#  -0.01608679 -0.01525714 -0.02534399]
# ranked
# [ 7  6  3  0  1  2  4  8  5  9 13 11 12 14 10]
# weight
# [ 0.03027158  0.02944122  0.02100642  0.0466252   0.01473765  0.01230972
#   0.05677486  0.08102311  0.01446427  0.00632119 -0.02152517 -0.00379137
#  -0.00407784  0.00101088 -0.00530084]
# ranked
# [ 7  6  3  0  1  2  8  5  4  9 11 13 12 14 10]
# weight
# [ 0.01821793  0.01646602  0.01074347  0.03995023  0.00250348  0.00312992
#   0.04543218  0.06934124  0.00663548 -0.00648229 -0.02624685 -0.01500524
#  -0.0160485  -0.01518648 -0.02520969]

In [20]:
# import time
# a = np.array([[1,1,1],[2,2,2], [3,3,3], [0,0,0]])
# start = time.time()
# c = pair_dist(a, metric='manhattan')
# print(time.time()-start)
# print(c,'\n')
# start = time.time()
# d = pdist(a, metric='cityblock')
# f = squareform(d)
# print(time.time()-start)
# print(d)
# print(f)

In [21]:
# from sklearn.datasets import load_iris, load_digits, load_wine

# X, Y = load_iris(return_X_y= True)
# print(X.shape, Y.shape)
# ranked, weight= Modified_MultiSurf_configure(X,Y,prior='uniform', numUpdates='all', categoricalX='off', kernelWidth = 3)

# print('ranked')
# print(ranked)
# print('weight')
# print(weight)

In [22]:
# X= np.array([[-2,-1],[-1,0],[0,1],[1,2],[2,4],[3,6],[6,8],[4,5],[7,7],[8,9],[9,10],[10,11],[11,12],[12,13]])
# X= np.array([[-2,-1],[-1,0],[0,1],[1,2],[2,4],[3,6],[4,8],[6,5],[7,7],[8,9],[9,10],[10,11],[11,12],[12,13]])
X= np.array([[.8,1.2],[1.9,1.4],[3,1.8],[4.3,2.9],[4.4,4],[4.7,5.3],[5.3,7.3],[5,6.5],[5.5,8],[5.9,9.9],[6.1,10.8],[7.2,11.5],[8.2,12],[9.8,12.3]])
Y= np.array([1,1,1,1,1,1,1,2,2,2,2,2,2,2])
ranked, weight =Modified_MultiSurf_configure(X,Y,prior='uniform', numUpdates='all', categoricalX='off', kernelWidth = 3)
print(ranked, weight)

[1 0] [-0.12702534 -0.1031529 ]
